This create one spatiotemporal diagram for one incident

To generate the diagrams, first fill in the fromIncidentToDataInterpolationData.ipynb. 
Look for the global variables, since they might need to be adjusted based on the speed flow dataset which is being used. 

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

In [7]:
# could later set global variables here
minutes = 240  # 20 hours
meters = 6000 # 10 km

minute_steps = 10 
meter_steps = 100

sigma = 200 # standard was 45
tau = 45 # standard was 45

folders_to_generate = ["/Users/vree/GithubRepos/DataScience/basFiles/3hoursFromIncidentToInterpolationData copy/noIncidents/incidentInterpolationData/intensiteit-snelheid-rotterdam-1-7-Maart",
                        "/Users/vree/GithubRepos/DataScience/basFiles/3hoursFromIncidentToInterpolationData copy/noIncidents/incidentInterpolationData/intensiteit-snelheid-rotterdam-8-14-Maart",
                        "/Users/vree/GithubRepos/DataScience/basFiles/3hoursFromIncidentToInterpolationData copy/noIncidents/incidentInterpolationData/intensiteit-snelheid-rotterdam-15-21-Maart",
                        "/Users/vree/GithubRepos/DataScience/basFiles/3hoursFromIncidentToInterpolationData copy/noIncidents/incidentInterpolationData/intensiteit-snelheid-rotterdam-22-29-Maart"]

In [8]:
import os

In [9]:
def phi_0(x, t, xi, ti, sigma, tau):
    return np.exp(- (np.abs(x - xi) / sigma + np.abs(t - ti) / tau))

def N(x, t, datapoints, sigma, tau):
    return sum(phi_0(x, t, xi, ti, sigma, tau) for xi, ti, _ in datapoints)

def V(x, t, datapoints, sigma, tau):
    normalization_factor = N(x, t, datapoints, sigma, tau)
    return sum(phi_0(x, t, xi, ti, sigma, tau) * vi for xi, ti, vi in datapoints) / normalization_factor

# takes in datapoints, which is a list of tuples of the form (x, t, v) for each datapoint
def spatiotemporal(datapoints):
    

    # Create a grid for the spatiotemporal map
    spatio_map = np.zeros((int(minutes / minute_steps), int(meters / meter_steps)))

    for i in range(0, minutes, minute_steps):
        for j in range(0, meters, meter_steps):
            spatio_map[int(i / minute_steps), int(j / meter_steps)] = V(j, i, datapoints, sigma, tau)

    return spatio_map

# Example synthetic data (xi, ti, vi)
# datapoints = [(np.random.randint(0, meters), np.random.randint(0, minutes), np.random.uniform(20, 120)) for _ in range(100)]
def create_spatio_diagram(x_t_v_data_path, incident_id, max_length = meters):

    data1  =pd.read_csv(x_t_v_data_path)

    # print minimum of values of the data for each column
    print(data1.min())
    print(data1.max())

    datapoints = [(row[0], row[1], row[2]) for index, row in data1.iterrows()]

    # Generate the spatiotemporal map
    spatio_map = spatiotemporal(datapoints)

    print("Contains NaN:", np.isnan(spatio_map).any())
    print("Contains Inf:", np.isinf(spatio_map).any())

    # Define the axes
    y = np.linspace(0, max_length, spatio_map.shape[1])  # Location axis for the y-axis
    x = np.linspace(0, minutes, spatio_map.shape[0])  # Time axis for the x-axis

    # Create the meshgrid for plotting, now with time on the x-axis and location on the y-axis
    T, Y = np.meshgrid(x, y)

    # Visualize the spatiotemporal map with location on the y-axis and time on the x-axis
    plt.figure(figsize=(15, 8))
    cp = plt.contourf(T, Y, spatio_map.T, levels=np.linspace(spatio_map.min(), spatio_map.max(), 100), cmap='viridis')
    plt.colorbar(cp, label='V (km/h)')



    # Set the labels correctly
    plt.xlabel('Time (minutes)')
    plt.ylabel('Location (meters)')
    plt.title('Spatiotemporal Traffic Data Visualization')
    # set in the subtitle the settings for sigma and tau
    plt.suptitle(f'Sigma = {sigma}, Tau = {tau}')
    
    sub_folder = x_t_v_data_path.split("/")[-2]
    # safe figure to ./spatiodiagrams folder, with random name
    # if folder does not exist create it
    base_path = '/Users/vree/GithubRepos/DataScience/basFiles/3hoursFromIncidentToInterpolationData copy/noIncidents/diagramsForModelTraining/' + sub_folder
    if not os.path.exists(base_path):
        os.makedirs(base_path)
    
    
    plt.savefig('/Users/vree/GithubRepos/DataScience/basFiles/3hoursFromIncidentToInterpolationData copy/noIncidents/diagramsForModelTraining/' + sub_folder + "/" + incident_id + '.png')







In [10]:
for folder_to_generate in folders_to_generate:
    # create all the spatio diagrams for the incidents
    # get incident data from ./incidentInterpolationData folder and create spatio diagrams
    min_max_df = pd.DataFrame(columns=['incident_id', 'min_x', 'max_x'])

    for filename in os.listdir(folder_to_generate):
        if filename.endswith(".csv"):
            print(filename)
            # get full path to the file
            filepath = folder_to_generate + "/" + filename
            df = pd.read_csv(filepath)
            min_x = df['x'].min()
            max_x = df['x'].max()
            print(f"minimum x for incident {filename} is {min_x}")
            print(f"maximum x for incident {filename} is {max_x}")

            # Add to min_max_df
            min_max_df = pd.concat([min_max_df, pd.DataFrame({
                'incident_id': [filename.split('.')[0]],
                'min_x': [min_x],
                'max_x': [max_x]
            })], ignore_index=True)
            
            
            incident_id = filename.split('.')[0]
            # set to filepath
            x_t_v_data_path = folder_to_generate + "/" + filename
            
            if max_x < 6000:
                create_spatio_diagram(x_t_v_data_path, incident_id, max_x)
            else:
                create_spatio_diagram(x_t_v_data_path, incident_id)
            print("finished")
        else:
            continue
        

# say program finished
os.system('say "full spatio finished"')

no_incident_camera_baseRWS01_MONIBAS_0201hrr0319ra_2020-03-02 05:33:42.csv
minimum x for incident no_incident_camera_baseRWS01_MONIBAS_0201hrr0319ra_2020-03-02 05:33:42.csv is 431.0565900799224
maximum x for incident no_incident_camera_baseRWS01_MONIBAS_0201hrr0319ra_2020-03-02 05:33:42.csv is 6311.120786673314
x    431.05659
t      0.30000
v      7.00000
dtype: float64
x    6311.120787
t     239.300000
v     125.000000
dtype: float64


/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_74463/4187144941.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  min_max_df = pd.concat([min_max_df, pd.DataFrame({
/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_74463/1568644145.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  datapoints = [(row[0], row[1], row[2]) for index, row in data1.iterrows()]


Contains NaN: False
Contains Inf: False
finished
no_incident_camera_baseRWS01_MONIBAS_0160vwx0193ra_2020-03-06 02:20:49.csv
minimum x for incident no_incident_camera_baseRWS01_MONIBAS_0160vwx0193ra_2020-03-06 02:20:49.csv is 407.44839244929426
maximum x for incident no_incident_camera_baseRWS01_MONIBAS_0160vwx0193ra_2020-03-06 02:20:49.csv is 4067.66509401913
x    407.448392
t      0.183333
v      0.000000
dtype: float64
x    4067.665094
t     239.183333
v     141.000000
dtype: float64


/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_74463/1568644145.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  datapoints = [(row[0], row[1], row[2]) for index, row in data1.iterrows()]


Contains NaN: False
Contains Inf: False
finished
no_incident_camera_baseRWS01_MONIBAS_0160vwx0193ra_2020-03-05 02:22:51.csv
minimum x for incident no_incident_camera_baseRWS01_MONIBAS_0160vwx0193ra_2020-03-05 02:22:51.csv is 407.44839244929426
maximum x for incident no_incident_camera_baseRWS01_MONIBAS_0160vwx0193ra_2020-03-05 02:22:51.csv is 4479.8323923520575
x    407.448392
t      0.150000
v      0.000000
dtype: float64
x    4479.832392
t     239.150000
v     134.000000
dtype: float64


...

/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_74463/1568644145.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  datapoints = [(row[0], row[1], row[2]) for index, row in data1.iterrows()]


Contains NaN: False
Contains Inf: False
finished
no_incident_camera_baseRWS01_MONIBAS_0150vws0507ra_2020-03-24 00:08:14.csv
minimum x for incident no_incident_camera_baseRWS01_MONIBAS_0150vws0507ra_2020-03-24 00:08:14.csv is 55.52589440530437
maximum x for incident no_incident_camera_baseRWS01_MONIBAS_0150vws0507ra_2020-03-24 00:08:14.csv is 5993.323397221873
x    55.525894
t     0.766667
v    15.000000
dtype: float64
x    5993.323397
t     239.766667
v     189.000000
dtype: float64


/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_74463/1568644145.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  datapoints = [(row[0], row[1], row[2]) for index, row in data1.iterrows()]


Contains NaN: False
Contains Inf: False
finished
no_incident_camera_baseRWS01_MONIBAS_0150vws0507ra_2020-03-23 03:27:14.csv
minimum x for incident no_incident_camera_baseRWS01_MONIBAS_0150vws0507ra_2020-03-23 03:27:14.csv is 725.0619709119824
maximum x for incident no_incident_camera_baseRWS01_MONIBAS_0150vws0507ra_2020-03-23 03:27:14.csv is 5458.955875199852
x    725.061971
t      0.766667
v     13.800000
dtype: float64
x    5458.955875
t     239.766667
v     185.000000
dtype: float64


/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_74463/1568644145.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  datapoints = [(row[0], row[1], row[2]) for index, row in data1.iterrows()]


Creating maps without interpolation below

In [ ]:

# # show map without extrapolating the data, just the traces of the cameras

# def spatiotemporal_unique(datapoints):
#     # Create a grid for the spatiotemporal map
#     spatio_map = np.zeros((int(minutes / minute_steps)+2, int(meters / meter_steps)+2))
    
#     # per location the datapoints are added chronologically, therefore we can use here the minutesteps as well
#     for i in range(0, len(datapoints), minute_steps):
#         spatio_x_axis = int(datapoints[i][1]/minute_steps)
#         spatio_y_axis = int(datapoints[i][0]/meter_steps)
#         print("spatio x axis: ", spatio_x_axis, "for i = ", i)
#         print("spatio y axis: ", spatio_y_axis, "for i = ", i)
#         spatio_map[spatio_x_axis, spatio_y_axis] = V(datapoints[i][0], datapoints[i][1], datapoints, sigma, tau)
        
#     return spatio_map


# # Example synthetic data (xi, ti, vi)
# # datapoints = [(np.random.randint(0, meters), np.random.randint(0, minutes), np.random.uniform(20, 120)) for _ in range(100)]

# # Load the data
# data = pd.read_csv(x_t_v_data_path)
# datapoints = [(row[0], row[1], row[2]) for index, row in data.iterrows()]

# # Generate the spatiotemporal map
# spatio_map = spatiotemporal_unique(datapoints)

# # Define the axes
# y = np.linspace(0, meters, spatio_map.shape[1])  # Location axis for the y-axis
# x = np.linspace(0, minutes, spatio_map.shape[0])  # Time axis for the x-axis

# # Create the meshgrid for plotting, now with time on the x-axis and location on the y-axis
# T, Y = np.meshgrid(x, y)

# # Visualize the spatiotemporal map with location on the y-axis and time on the x-axis
# plt.figure(figsize=(15, 8))
# cp = plt.contourf(T, Y, spatio_map.T, levels=np.linspace(spatio_map.min(), spatio_map.max(), 100), cmap='viridis')
# plt.colorbar(cp, label='V (km/h)')

# # Set the labels correctly
# plt.xlabel('Time (minutes from 7 AM)')
# plt.ylabel('Location (km)')
# plt.title('Spatiotemporal Traffic Data Visualization')

# #save figure to ./spatiodiagrams_traces folder, with random name
# plt.savefig('./spatiodiagrams_traces/spatiotemporal' + str(random) + '.png')

# plt.show()      

# os.system('say "your program has finished"')
        
        


In [ ]:
# import os
# # For macOS
# os.system('say "your program has finished"')


